In [4]:
import numpy as np
import tensorflow as tf

In [2]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

(25, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]] 25
(25, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]


In [5]:
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  204152.19 
Prediction:
 [[-269.49643]
 [-327.29736]
 [-320.73468]
 [-348.6812 ]
 [-251.11163]
 [-184.84055]
 [-259.33212]
 [-189.75748]
 [-306.04877]
 [-282.58582]
 [-251.50699]
 [-249.33502]
 [-331.9882 ]
 [-277.1399 ]
 [-260.9839 ]
 [-331.25433]
 [-265.7511 ]
 [-309.92093]
 [-316.08185]
 [-282.40173]
 [-304.7035 ]
 [-306.30887]
 [-290.40454]
 [-273.1849 ]
 [-338.8998 ]]
10 Cost:  19.963753 
Prediction:
 [[152.38089]
 [179.77776]
 [178.8854 ]
 [195.44621]
 [135.61743]
 [100.28608]
 [147.96626]
 [112.18815]
 [167.85707]
 [156.37502]
 [140.9058 ]
 [137.60123]
 [184.18544]
 [151.94807]
 [148.18962]
 [183.30913]
 [142.90028]
 [178.8749 ]
 [176.01268]
 [157.9043 ]
 [172.9198 ]
 [169.39766]
 [164.96927]
 [152.16109]
 [186.62065]]
20 Cost:  10.183976 
Prediction:
 [[155.28586 ]
 [183.28598 ]
 [182.33319 ]
 [199.20482 ]
 [138.29413 ]
 [102.267136]
 [150.78049 ]
 [114.27508 ]
 [171.14413 ]
 [159.4293  ]
 [143.61836 ]
 [140.28467 ]
 [187.74399 ]
 [154.90211 ]
 [151.0197  ]
 [186.87111 

In [6]:
print("Your score will be ", sess.run(
    hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ", sess.run(hypothesis,
                                        feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Your score will be  [[181.7969]]
Other scores will be  [[180.0652]
 [177.8908]]


만약 데이터의 크기가 너무 커서 하나의 np에 다 넣을수 없는경우

텐서플로우에서 제공하는 Queue Runners를 사용하면됨

간단히 설명하면 여러개의 file들을 랜덤하게 셔플한 후 queue에 file들을 쌓는다

그리고 몇개의 reader들로 읽어오고 decoder들로 양식에 맞게 바꾸고

다시 이 데이터를 queue에 쌓고 이것들을 일정량만큼 학습시킨다

In [7]:
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

0 Cost:  166847.22 
Prediction:
 [[-244.28423]
 [-288.4739 ]
 [-286.79776]
 [-313.95547]
 [-217.29965]
 [-161.88899]
 [-238.68729]
 [-182.29314]
 [-270.2714 ]
 [-253.30153]]
10 Cost:  36.464622 
Prediction:
 [[149.76138]
 [185.20679]
 [179.88718]
 [194.32536]
 [143.9565 ]
 [104.49854]
 [141.79587]
 [ 99.79575]
 [172.46542]
 [156.84357]]
20 Cost:  28.78478 
Prediction:
 [[152.41754 ]
 [188.39777 ]
 [183.03123 ]
 [197.75948 ]
 [146.38148 ]
 [106.3017  ]
 [144.38177 ]
 [101.734245]
 [175.4595  ]
 [159.63884 ]]
30 Cost:  28.640997 
Prediction:
 [[152.43254 ]
 [188.41376 ]
 [183.04724 ]
 [197.78679 ]
 [146.38504 ]
 [106.31932 ]
 [144.41747 ]
 [101.782104]
 [175.48605 ]
 [159.68509 ]]
40 Cost:  28.497791 
Prediction:
 [[152.42975]
 [188.40833]
 [183.04214]
 [197.79109]
 [146.37228]
 [106.32485]
 [144.43591]
 [101.81712]
 [175.49252]
 [159.71266]]
50 Cost:  28.355597 
Prediction:
 [[152.42686 ]
 [188.40277 ]
 [183.03694 ]
 [197.7952  ]
 [146.3594  ]
 [106.330246]
 [144.45416 ]
 [101.85194 ]
 

In [8]:
# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Your score will be  [[193.0099]]
Other scores will be  [[156.29436]
 [180.12758]]
